In [334]:
from plotnine import *
import pandas
from sklearn import preprocessing
import itertools
from statistics import mean
from solarized import *
pandas.options.display.max_rows=10

Index(['index', 'bytes_memory', 'cpu_percent', 'bytes_sent', 'bytes_recv',
       'wall_time', 'exitcode', 'db_time', 'rep', 'net', 'scale', 'wflow',
       'optimizer'],
      dtype='object')

In [ ]:
# 1.{5end_to_end} tpch 1,2,3,4,5 with scale 1, 10
# 2 {5module4} and 3. module4 | also {5module4mem} and {5module4net}
# 3.{5tpchmodin} tpch 1, 4, 5 with modin on scale 10
# 4.{5micro_scales} micros with scale 1, 10 | also {5micro_traffic}
# 5.{5micro_net} micro join, selection with net=wan and scale 1

In [ ]:
factors= ['wflow', 'optimizer', 'net', 'scale']
df=pandas.read_feather('specific-reposts/5end_to_end.feather')
df.columns

# CHECK

In [335]:
df['exitcode'].value_counts()

0    186285
1     21940
Name: exitcode, dtype: int64

In [336]:
df.query("exitcode == 1")['wflow'].unique()

array(['tpch1.py', 'q07.sql', 'q08.sql', 'q09.sql'], dtype=object)

# PREPROCESS

In [399]:
overview = df.groupby(by=factors).first().reset_index()
overview.columns

Index(['wflow', 'optimizer', 'net', 'scale', 'index', 'bytes_sent',
       'wall_time', 'db_time', 'gb_memory', 'cpu', 'gb_net',
       'procedural_time'],
      dtype='object')

In [ ]:
overview = overview["wflow", "optimizer", "scale", "shared_wall_time"]]

# PROCEDURAL AND DB TIME

In [402]:
def prep_for_plot(origin_df: pandas.DataFrame, measurements:list):
    prj = origin_df[factors+measurements]
    return prj.melt(id_vars=factors, value_vars=measurements,
                    value_name='measurement', var_name='var')


In [ ]:
(
        ggplot(plot_df, aes('optimizer', y='measurement', fill='var'))
        + geom_col(width=0.3)
        #+ scale_fill_manual(values=cdict) 
        + facet_wrap('wflow', scales='free')
        + ggtitle(plot_title)
        + xlab("Optimization Method")
        + ylab("Time (in sec.)")
        + theme(figure_size=(5, 2.5),
               subplots_adjust={'hspace': 1, 'wspace': 0.25})
    )

In [ ]:
curplot = plot_f(qdf, query)
curplot.save(f"plots/{folder}/{name}.png")
display(curplot)

In [420]:
# pandas.options.display.max_rows=200
# exec_time_melt.query("optimizer=='optimized' and net=='loc' and scale=='1' and index=='false'")

In [421]:
df_time =prep_for_plot(overview, ['procedural_time', 'db_time'])
df_net = prep_for_plot(overview, ['gb_net'])
df_cpu = prep_for_plot(interproc, ['cpu'])
df_mem = prep_for_plot(interproc, ['gb_net'])


In [422]:
all_factors = cycle_factors()

In [425]:
gen_plots(all_factors)

net == 'lan' and scale == '1' and index == 'false'


E:\ssdapps\anaconda\lib\site-packages\plotnine\ggplot.py:719: PlotnineWarning: Saving 8 x 2.5 in image.
E:\ssdapps\anaconda\lib\site-packages\plotnine\ggplot.py:722: PlotnineWarning: Filename: plots/plot_mem/lan1false.png


SyntaxError: unexpected EOF while parsing (<string>, line 1)

In [56]:
# variable_cat = pandas.CategoricalDtype(categories=['python_time', 'db_time'], ordered=True)
# df['variable'] = df['variable'].astype(variable_cat)
# cdict={'python_time': C_BLUE,
#        'db_time': C_GREEN
# }
# df.rename(columns={'value':'Execution time %'}, inplace=True)

In order to explain these results, we also ran microbenchmarks: workflows consisting of a single operation on the dataset. The following list provides a quick overview of the microbenchmarks, all of which were expressed in pandas:
- join (Two database tables are pulled from the RDBMS and an inner join is performed)
- max (A database table is pulled from the RDBMS and a maximum of a column is calculated)
- projection (A database table is pulled from the RDBMS and a subset of the columns are removed inside the python runtime)
- selection (A database table is pulled from the RDBMS and a subset of the rows are removed inside the python runtime)
For a detailed look at the microbenchmarks, one can see the workflows inside the "benchmarks" folder in the project's git repositoty. It should be noted that none of the microbenchmarks modify tables inside the RDBMS.

Since the various microbenchmark workflows have different base runtimes (e.g. aggregations were 10 times faster in this specific measurement), the values we see in the above graphic are normalized, such that the slowest optimization method has a value of 1. Lower values are better.  We can see that our optimizer generally performed better than both modin and standard pandas "base".

To gain better insight about this, we split the overall wall time between the time it took the database to deliver the queried data "DB time" and the time it took the python environment to execute the operation "Python time". As expected, our performing operator pushdown to RDBMS practically eliminates "Python time". What is arguably more interesting, is that in most cases performing the pushdown also decreased "DB time". 